In [1]:
import numpy as np
import matplotlib.pyplot as plt
import librosa

%load_ext autoreload
%autoreload 2

In [2]:
from common.multichannel_spectrograms import *

# STFT Log-spacing Buildup

In [3]:
win_length = 255
hop_length = 64

In [4]:
is_odd = win_length % 2 == 1
lin_spacing = 1 / win_length
idx0 = win_length // 2
lin_freqs = np.concatenate([
    np.linspace(-0.5, 0, win_length // 2 + 1, endpoint=True),
    np.linspace(lin_spacing, 0.5, (win_length - 1) // 2, endpoint=is_odd)
])
neg_log_freqs = [-(10 ** i) for i in np.linspace(-0.3010299956639812, -3, win_length // 2, endpoint=True)]
log_freqs = np.array(neg_log_freqs + [0] + [-f for f in neg_log_freqs[not is_odd:][::-1]])
lr_freqs = librosa.fft_frequencies(sr=1, n_fft=win_length)
print(idx0)

In [5]:
plt.plot(lin_freqs)
plt.plot(log_freqs)
plt.plot(lr_freqs, ':')
plt.plot([0, win_length], [0.5, 0.5], 'k--')
print(lin_freqs.shape, log_freqs.shape, lr_freqs.shape)
print(lin_freqs[-1], log_freqs[-1], lr_freqs[-1])

In [6]:
plt.plot(lin_freqs[idx0:])
plt.plot(log_freqs[idx0:])
plt.plot(lr_freqs, ':')
plt.yscale('log')
print(lin_freqs[idx0], log_freqs[idx0], lr_freqs[0])

In [7]:
plt.plot(lin_freqs[idx0:])
plt.plot(-lin_freqs[:idx0:-1])
plt.plot(log_freqs[idx0:])
plt.plot(-log_freqs[:idx0:-1])
plt.plot(lr_freqs, ':')
plt.yscale('log')
print(lin_freqs[idx0], log_freqs[idx0], lr_freqs[0])

In [8]:
omega = np.exp(-2j * np.pi / win_length)
Wlin = omega ** (np.outer(np.fft.ifftshift(lin_freqs) * win_length, np.arange(win_length))) #/ np.sqrt(win_length)
Wlog = omega ** (np.outer(np.fft.ifftshift(log_freqs) * win_length, np.arange(win_length))) #/ np.sqrt(win_length)

def lin_dftmtx(N):
    is_odd = N % 2 == 1
    spacing = 1 / N
    freqs = np.concatenate([
        np.linspace(-0.5, 0, N // 2 + 1, endpoint=True),
        np.linspace(spacing, 0.5, (N - 1) // 2, endpoint=is_odd)
    ])
    omega = np.exp(-2j * np.pi / N)
    W = omega ** (np.outer(np.fft.ifftshift(freqs) * N, np.arange(N))) / np.sqrt(N)
    return freqs, W


def log_dftmtx(N, min_exponent=-3):
    is_odd = N % 2 == 1
    neg_freqs = [-(10 ** i) for i in np.linspace(np.log10(0.5), -3, N // 2, endpoint=True)]
    freqs = np.array(neg_freqs + [0] + [-f for f in neg_freqs[not is_odd:][::-1]])
    omega = np.exp(-2j * np.pi / N)
    W = omega ** (np.outer(np.fft.ifftshift(freqs) * N, np.arange(N))) / np.sqrt(N)
    return freqs, W


def fft2rfft(X):
    N = X.shape[0]
    X = X.reshape(N, -1)
    is_odd = N % 2 == 1
    is_even = N % 2 == 0
    split_idx = N // 2 + is_odd
    rX = X[:split_idx, :]
    rX[1:, :] += X[split_idx + is_even:, :][::-1, :].conjugate()
    return rX.squeeze()

log_freqs, Wlog = log_dftmtx(win_length)
lin_freqs, Wlin = lin_dftmtx(win_length)


In [9]:
plt.imshow(Wlin.real)
plt.colorbar()

In [10]:
plt.imshow(Wlog.real)
plt.colorbar()

In [11]:
plt.imshow(np.abs(Wlin @ np.conjugate(Wlin.T)))
plt.colorbar()

In [12]:
plt.imshow(np.angle(Wlin), aspect='equal')

In [13]:
plt.imshow(np.angle(Wlog), aspect='equal')

In [14]:
x = np.sin(2 * np.pi * 50 * np.arange(win_length) / win_length) + 0.5
plt.plot(x)

In [15]:
plt.plot(np.hanning(win_length))
plt.plot(np.abs(np.fft.fftshift(np.fft.fft(np.hanning(win_length))) / np.sqrt(win_length)))

In [16]:
window = np.hanning(win_length)
Xlin = Wlin @ (window * x)
Xlog = Wlog @ (window * x)
Xfft = np.fft.fft(window * x) / np.sqrt(win_length)
aXlin = np.fft.fftshift(np.abs(Xlin))
aXlog = np.fft.fftshift(np.abs(Xlog))
aXfft = np.fft.fftshift(np.abs(Xfft))
plt.plot(lin_freqs, aXlin)
plt.plot(log_freqs, aXlog)
plt.plot(lin_freqs, aXfft, ':')

In [17]:
arXlin = np.abs(fft2rfft(Xlin))
arXlog = np.abs(fft2rfft(Xlog))
arXfft = np.abs(fft2rfft(Xfft))
plt.plot(lin_freqs[idx0:], arXlin)
plt.plot(log_freqs[idx0:], arXlog)
plt.plot(lin_freqs[idx0:], arXfft, ':')

In [18]:
plt.plot(arXlog)

In [19]:
xx = x = np.sin(2 * np.pi * 50 * np.arange(win_length * 10) / win_length +
                2 * np.pi * 20 * (np.arange(win_length * 10) / win_length) ** 2) + 0.5

xx_strided = np.lib.stride_tricks.sliding_window_view(xx, win_length)[::hop_length]
print(xx_strided.shape)
plt.imshow(xx_strided, aspect='auto', interpolation='none')

In [20]:
STFTlin = lin_rstft(xx, win_length, hop_length, window='hann')[1]
plt.imshow(np.abs(STFTlin), aspect='auto', interpolation='none', origin='lower')
print(STFTlin.shape)

In [21]:
STFTlog = log_rstft(xx, win_length, hop_length, window='hann')[1]
plt.imshow(np.abs(STFTlog), aspect='auto', interpolation='none', origin='lower')
print(STFTlog.shape)

# EEG Image Params

In [22]:
from signal_diffusion.data.parkinsons import ParkinsonsPreprocessor

In [23]:
pp = ParkinsonsPreprocessor("/mnt/d/data/signal-diffusion/parkinsons/", 2000)

In [24]:
x = pp.get_data("sub-002")

In [28]:
N = 512
hop = 128
spec1 = multichannel_spectrogram(x[:, :3000], N, hop, N, bin_spacing="log")
spec2 = multichannel_spectrogram(x[:, :3000], N, hop, N, bin_spacing="linear")

In [29]:
stft1 = log_rstft(x[0, :4000], N * 2 - 1, hop, 'hann')[1]
stft2 = librosa.stft(y=x[0, :4000], win_length=N, hop_length=hop, n_fft=N * 2 - 1,)
print(stft1.shape)
print(stft2.shape)

In [30]:
fig, axs = plt.subplots(1, 2, figsize=(10, 5))
axs[0].imshow(spec1, aspect='auto', interpolation='none', origin='upper')
axs[1].imshow(spec2, aspect='auto', interpolation='none', origin='upper')